In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import csv
import time
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from geopy import distance
#import preprocess as pre

In [2]:
train = pd.read_csv("ais_train.csv", sep='|')
test = pd.read_csv("ais_test.csv")
train['time'] = pd.to_datetime(train['time'])
test['time'] = pd.to_datetime(test['time'])

In [3]:
def singleBoatCleanup(boat: pd.DataFrame, removeID: bool) -> pd.DataFrame:
    """ 
    Input a Pandas dataframe of a single boat and output a cleaned version of the dataframe.
    """    
    #0-index everything
    boat.reset_index(drop=True, inplace=True)
    boat["time"] = boat["time"].astype("datetime64[ns]")

    if removeID:
        boat = boat.drop(columns=["vesselId", "portId"])

    boat["drift"] = (boat["cog"] - boat["heading"]+180)%360-180

    # --- Find last time we left a port ---
    #Figure out when navstat changes (2pac)
    boat["at_port"] = boat["navstat"].apply(lambda stat: True if stat == 5 else False)
    boat["change"] = boat["at_port"] != boat["at_port"].shift(-1)

    #Pick out rows with navstat == 5 and change == True
    wallah = boat[boat["navstat"] == 5]
    wallah = wallah[wallah["change"] == True]
    indices = wallah.index

    #Caveman fix
    boat["timestamp_last_port"] = np.nan

    for i in indices:
        boat.at[i, "timestamp_last_port"] = boat.at[i, "time"]
    boat["timestamp_last_port"] = boat["timestamp_last_port"].ffill()

    origin = boat.at[0, "time"]
    boat["timestamp_last_port"] = boat["timestamp_last_port"].fillna(origin)

    #Insh'allah
    boat["time_at_sea"] = boat["time"]-boat["timestamp_last_port"]
    boat["time_at_sea"] = boat["time_at_sea"].apply(lambda x: x if x >= pd.Timedelta(0) else pd.Timedelta(0))
    boat = boat.drop(columns=["change", "timestamp_last_port"])

    # --- etaRaw fixing ---
    boat["etaRaw"] = boat["etaRaw"].apply(lambda eta: "2024-" + eta)
    boat["etaRaw"] = pd.to_datetime(boat["etaRaw"], errors='coerce') #Set errors to NaT
    boat = boat.dropna(subset=["etaRaw"])

    boat["etaRaw"] = boat["etaRaw"].apply(lambda eta: pd.to_datetime(eta))

    # --- Fixing pd.timestamp and pd.timedelta to numerical ---
    """ boat["time"] = boat["time"].astype("int64") // (10**9)
    boat["etaRaw"] = boat["etaRaw"].astype("int64") // (10**9)
    boat["time_at_sea"] = boat["time_at_sea"].dt.total_seconds() """
    boat["dist_s_l"] = 0
    boat["tot_dist"] = 0
    boat["time_s_l"] = 0
    boat["speed"] = 0
    k = 0
    boat = boat.reset_index(drop = True)
    for _,i in boat.iterrows():
        if k==0:
            k+=1
            j=i
        else:
            min1 = j
            min2 = i
            dist = distance.geodesic((min2['latitude'],min2['longitude']),(min1['latitude'],min1['longitude'])).km
            boat.at[k,"dist_s_l"] = dist
            boat.at[k,"tot_dist"] = boat.at[k-1,"tot_dist"] + dist
            tdddd = abs(j['time'].timestamp()-i['time'].timestamp())
            boat.at[k,"time_s_l"] = tdddd
            speed = dist/tdddd*3600
            if speed > 75:
                boat.at[k-1,"speed"] = np.nan
                boat.at[k,"speed"] = 26
            else: boat.at[k,"speed"] = speed
            k+=1
            j=i
    boat=boat.dropna(subset=["speed"])
    boat["speed_lag"] = boat["speed"].shift(1)
    boat=deltafier(boat)
    return boat

def deltafier(df):
    data = df.copy()
    data["delta_lat"] = data["latitude"]-data["latitude"].shift(1)
    data["delta_lon"] = (data["longitude"]-data["longitude"].shift(1)+180)%360-180
    data["delta_time"] = timefix(data)["time"]-timefix(data)["time"].shift(1)
    data["last_lon"] = data["longitude"].shift(1)
    data["last_lat"] = data["latitude"].shift(1)
    return data[1:]

def timefix(df):
    data = df.copy()
    time3 = []
    for i in data['time']:
        time3.append(int(i.timestamp()-1704067200))
    data['time'] = np.array(time3, dtype = 'float32')
    return data

In [4]:
start = time.time()
uniqueVesselId = train["vesselId"].unique()
ais_train_processed_list = []

for id in uniqueVesselId:
        #Isolate a single boat and clean
    boat = train[train["vesselId"] == id]
    boat = singleBoatCleanup(boat, False)

        #Appending rows to list
    for _, row in boat.iterrows():
        ais_train_processed_list.append(row.to_dict())

cleandata = pd.DataFrame(ais_train_processed_list).sort_values("time").reset_index(drop=True)
print(time.time()-start)

837.9111707210541


In [5]:
#cleandata = pd.read_csv("ais_clean.csv",sep="|")
cleandata['time'] = pd.to_datetime(cleandata['time'])
cleandata.head()

time    cog   sog  rot  heading  navstat              etaRaw  \
0 2024-01-01 00:17:17   73.9   0.1   12      120        1 2024-12-28 05:00:00   
1 2024-01-01 00:18:57  150.7   0.1    0      251        0 2024-12-31 04:00:00   
2 2024-01-01 00:19:39   56.4  12.9  -16       56        0 2024-12-30 20:00:00   
3 2024-01-01 00:20:35  114.5   0.0    0      275        5 2024-01-03 13:30:00   
4 2024-01-01 00:21:37  210.4   0.0   -5      122        1 2024-12-31 04:00:00   

   latitude  longitude                   vesselId  ...  dist_s_l  tot_dist  \
0 -26.77586  153.23453   61e9f3acb937134a3c4bfe23  ...  0.033917  0.033917   
1  38.47387   15.91592   61e9f3cab937134a3c4bff01  ...  0.002618  0.002618   
2  33.90815  130.92404   61e9f3e1b937134a3c4bff59  ...  2.085388  2.085388   
3  43.44237   -3.82309  clh6aqawa0002gh0zypfa5dut  ...  0.005775  0.005775   
4  40.68658   29.31613   61e9f3c7b937134a3c4bfedf  ...  0.025431  0.025431   

   time_s_l      speed  speed_lag  delta_lat  delta_lon  delta_time  \
0       361   0.338229        0.0    0.00026    0.00018       361.0   
1       159   0.059274        0.0    0.00000   -0.00003       159.0   
2       334  22.477237        0.0    0.00412    0.02200       334.0   
3       720   0.028873        0.0   -0.00001    0.00007       720.0   
4       360   0.254307        0.0   -0.00021    0.00012       360.0   

    last_lon  last_lat  
0  153.23435 -26.77612  
1   15.91595  38.47387  
2  130.90204  33.90403  
3   -3.82316  43.44238  
4   29.31601  40.68679  

[5 rows x 24 columns]

In [6]:
#endrer tid og 1hoter vessel
warnings.filterwarnings('ignore')

def preprocess(df):
    data = df.copy()
    time3 = []
    for i in data['time']:
        time3.append(int(i.timestamp()-1704067200))
    data['time'] = np.array(time3, dtype = 'float32')
    data = pd.get_dummies(data, columns=['vesselId'], drop_first=True)
    return(data)

def timefix(df):
    data = df.copy()
    time3 = []
    for i in data['time']:
        time3.append(int(i.timestamp()-1704067200))
    data['time'] = np.array(time3, dtype = 'float32')
    return data

def speed(x,time_a,time_b):
    if time_a == time_b:
        return 0
    else:
        return min((x)/(time_b-time_a),50)
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
#Må brukes på enkeltbåt
def deltafier(df):
    data = df.copy()
    data["delta_lat"] = data["latitude"]-data["latitude"].shift(1)
    data["delta_lon"] = (data["longitude"]-data["longitude"].shift(1)+180)%360-180
    data["delta_time"] = timefix(data)["time"]-timefix(data)["time"].shift(1)
    data["last_lon"] = data["longitude"].shift(1)
    data["last_lat"] = data["latitude"].shift(1)
    return data[1:]

In [7]:
uniquevessel = [i for i in cleandata.copy()['vesselId'].unique()]
bettertraining = cleandata[0:1]
for i in uniquevessel:
    boat = cleandata[cleandata['vesselId'] == i]
    bettertraining.loc[len(bettertraining.index)] = boat.iloc[-1]
bettertraining = bettertraining[1:]
bettertraining.tail()

time    cog   sog  rot  heading  navstat  \
683 2024-05-02 02:19:56  286.5  13.7    0      288        0   
684 2024-05-04 13:45:52  199.3  18.5    0      200        0   
685 2024-04-29 00:09:09  320.4  16.5    6      322        8   
686 2024-05-04 05:24:33   59.8  14.9    0       48        0   
687 2024-05-05 07:04:09  229.9   0.0    0       17        5   

                 etaRaw  latitude  longitude                  vesselId  ...  \
683 2024-05-21 14:00:00  26.93694 -123.62850  61e9f45cb937134a3c4c022b  ...   
684 2024-05-05 20:00:00  36.64848  125.89047  61e9f3f7b937134a3c4bffc5  ...   
685 2024-05-08 02:00:00 -33.86094   17.43360  61e9f437b937134a3c4c013f  ...   
686 2024-05-14 15:00:00  37.22231  144.98452  61e9f47ab937134a3c4c02f7  ...   
687 2024-10-06 13:00:00  30.33982  -88.57598  61e9f3adb937134a3c4bfe37  ...   

      dist_s_l      tot_dist  time_s_l      speed  speed_lag  delta_lat  \
683  27.834818   6975.886422      4039  24.809444  25.568070    0.06341   
684   4.361289  32103.982895       474  33.123712  32.222610   -0.03816   
685  71.837231  36784.536255      8118  31.856865  31.741977    0.48988   
686   3.871926  20950.254956       504  27.656614  28.290900    0.01822   
687   0.000962      0.136219       537   0.006446   0.016949    0.00000   

     delta_lon  delta_time   last_lon  last_lat  
683   -0.27113      4039.0 -123.35737  26.87353  
684   -0.01167       474.0  125.90214  36.68664  
685   -0.50924      8118.0   17.94284 -34.35082  
686    0.03720       504.0  144.94732  37.20409  
687   -0.00001       537.0  -88.57597  30.33982  

[5 rows x 24 columns]

In [9]:
start = time.time()
trainingdata5 = cleandata[["delta_time","last_lat","last_lon","speed_lag","vesselId"]]
#trainingdata5 = pd.merge(trainingdata5,distancem,how="left",on="vesselId")
trainingdata5=trainingdata5.drop(columns="vesselId")
goaldata5 = cleandata[["delta_lat","delta_lon"]]
trainingdata5 = trainingdata5.reindex(sorted(trainingdata5.columns), axis=1)
print(time.time()-start)
trainingdata5.head()

0.053946495056152344


delta_time  last_lat   last_lon  speed_lag
0       361.0 -26.77612  153.23435        0.0
1       159.0  38.47387   15.91595        0.0
2       334.0  33.90403  130.90204        0.0
3       720.0  43.44238   -3.82316        0.0
4       360.0  40.68679   29.31601        0.0

In [10]:
start = time.time()
model5 = xgb.XGBRegressor(learning_rate=0.05,max_depth=20,n_estimators=3000)
#model5 = xgb.XGBRegressor()
model5.fit(trainingdata5,goaldata5)
print(time.time()-start)

607.2480084896088


In [11]:
uniquevessel = [i for i in test.copy()['vesselId'].unique()]
bettertraining = cleandata[0:1]
for i in uniquevessel:
    boat = cleandata[cleandata['vesselId'] == i]
    bettertraining.loc[len(bettertraining.index)] = boat.iloc[-1]
bettertraining = bettertraining[1:]
bettertraining.tail()

time    cog   sog  rot  heading  navstat  \
211 2024-04-26 13:50:08  188.2  17.1    2      187        0   
212 2024-04-30 18:39:21  251.9  14.3    9      253        0   
213 2024-04-30 07:08:07  137.5   4.5  -15      137        0   
214 2024-04-21 08:15:03    6.2  16.1    0        5        0   
215 2024-05-03 22:32:02   76.9  13.8    0       79        0   

                 etaRaw  latitude  longitude                  vesselId  ...  \
211 2024-04-27 20:00:00  36.66386  125.90680  631e3eb0192150e13fa41628  ...   
212 2024-05-03 09:30:00  34.64546  139.47670  61e9f41cb937134a3c4c0091  ...   
213 2024-04-29 14:00:00  32.79268  -79.92353  61e9f39fb937134a3c4bfdcd  ...   
214 2024-04-19 03:30:00  25.29443  121.26813  6326eed6c46d6a20d22ca319  ...   
215 2024-05-11 22:00:00   1.27494  104.27201  61e9f45ab937134a3c4c021b  ...   

      dist_s_l      tot_dist  time_s_l      speed  speed_lag  delta_lat  \
211   5.644802  42924.871134       630  32.256013  32.862252   -0.05054   
212   4.161817  55124.359833       558  26.850431  27.964774   -0.02140   
213   0.362835  55923.219291     48893   0.026716   0.004151   -0.00214   
214  11.355915  20291.132863      1488  27.473988  18.832745    0.09506   
215   6.056613  50329.787290       851  25.621396  24.054062    0.01101   

     delta_lon  delta_time   last_lon  last_lat  
211   -0.00715       630.0  125.91395  36.71440  
212   -0.03729       558.0  139.51399  34.66686  
213    0.00293     48893.0  -79.92646  32.79482  
214   -0.04219      1488.0  121.31032  25.19937  
215    0.05331       851.0  104.21870   1.26393  

[5 rows x 24 columns]

In [12]:
start = time.time()
test5 = test.copy()
uniqueVesselId = test5["vesselId"].unique()
test5_processed_list = []
for id in uniqueVesselId:
    boat = test5[test5["vesselId"] == id].reset_index(drop=True)
    last = bettertraining[bettertraining["vesselId"]==id].reset_index(drop=True)
    boat["delta_time"] = timefix(boat)["time"]-timefix(boat)["time"].shift(1)
    boat.at[0,"delta_time"] = timefix(boat).at[0,"time"]-timefix(last).at[0,"time"]
    boat.at[0,"last_lat"] = last.at[0,"latitude"]
    boat.at[0,"last_lon"] = last.at[0,"longitude"]
    boat.at[0,"speed_lag"] = last.at[0,"speed"]
        #Appending rows to list
    for _, row in boat.iterrows():
        test5_processed_list.append(row.to_dict())

test5_processed = pd.DataFrame(test5_processed_list)

print(time.time()-start)
test5_processed.head()

5.318640232086182


ID                  vesselId                time  scaling_factor  \
0    0  61e9f3aeb937134a3c4bfe3d 2024-05-08 00:03:16             0.3   
1  143  61e9f3aeb937134a3c4bfe3d 2024-05-08 00:36:14             0.3   
2  282  61e9f3aeb937134a3c4bfe3d 2024-05-08 00:51:12             0.3   
3  426  61e9f3aeb937134a3c4bfe3d 2024-05-08 01:12:14             0.3   
4  551  61e9f3aeb937134a3c4bfe3d 2024-05-08 01:30:14             0.3   

   delta_time  last_lat  last_lon  speed_lag  
0       900.0  31.14647 -81.49789   0.003178  
1      1978.0       NaN       NaN        NaN  
2       898.0       NaN       NaN        NaN  
3      1262.0       NaN       NaN        NaN  
4      1080.0       NaN       NaN        NaN

In [13]:
test5p = test5_processed.copy()
#test5p = pd.merge(test5p,bettertraining.copy(), on = "vesselId",how = "left")
#test5p['time']=test5p['time_x']
#test5p["delta_time"]=test5p["delta_time_x"]
#test5p["last_lat"]=test5p["last_lat_x"]
#test5p["last_lon"]=test5p["last_lon_x"]
test5p = test5p[["last_lat","last_lon","vesselId","time","delta_time","speed_lag"]]
#test5p = pd.merge(test5p,distancem,how="left",on="vesselId")
test5p = test5p.sort_values(by=["vesselId","time"])
#test5p = pd.get_dummies(test5p, columns=['navstat'], drop_first=True)
for column in trainingdata5:
    if column not in test5p:
        test5p[column] = False

test5p = test5p.reindex(sorted(test5p.columns), axis=1).reset_index()

test5p.head()

index  delta_time  last_lat  last_lon  speed_lag                time  \
0    697      1258.0   48.5332  -6.12003   1.730614 2024-05-08 00:12:27   
1    698      1620.0       NaN       NaN        NaN 2024-05-08 00:39:27   
2    699      3241.0       NaN       NaN        NaN 2024-05-08 01:33:28   
3    700      1078.0       NaN       NaN        NaN 2024-05-08 01:51:26   
4    701       723.0       NaN       NaN        NaN 2024-05-08 02:03:29   

                   vesselId  
0  61e9f38eb937134a3c4bfd8d  
1  61e9f38eb937134a3c4bfd8d  
2  61e9f38eb937134a3c4bfd8d  
3  61e9f38eb937134a3c4bfd8d  
4  61e9f38eb937134a3c4bfd8d

In [14]:
#lat,lon
start = time.time()
worker = test5p.copy().drop(columns="index")

def predi(data,vid,tim):
    work = data.copy().reset_index(drop=True)
    for i in range(len(work)-1):
        pred = model5.predict(work[i:i+1])[0]
        latlon = [work.at[i,"last_lat"]+pred[0],(work.at[i,"last_lon"]+pred[1]+180)%360-180]
        dist = distance.geodesic((work.at[i,"last_lat"],work.at[i,"last_lon"]),(latlon[0],latlon[1])).km
        work.at[i+1,"last_lat"]=latlon[0]
        work.at[i+1,"last_lon"]=latlon[1]
        work.at[i+1,"speed_lag"] = dist/work.at[i+1,"delta_time"]*3600
    last = model5.predict(work[-1:])[0]
    work["latitude"]=work["last_lat"].shift(-1)
    work["longitude"]=work["last_lon"].shift(-1)
    work.at[len(work)-1,"latitude"]=work.at[len(work)-1,"last_lat"]+last[0]
    work.at[len(work)-1,"longitude"]=(work.at[len(work)-1,"last_lon"]+last[1]+180)%360-180
    work["vesselId"]=id
    work["time"]=tim
    return work

uniqueVessel = worker["vesselId"].unique()
dog_list = []
for id in uniqueVessel:
    #Isolate a single boat and clean
    boat = worker[worker["vesselId"] == id].drop(columns=["vesselId"]).reset_index(drop=True)
    tim = boat["time"]
    boat = predi(boat.drop(columns="time"),id,tim)
    for _, row in boat.iterrows():
        dog_list.append(row.to_dict())

worker = pd.DataFrame(dog_list).sort_values(by=["vesselId","time"]).reset_index(drop=True)

bro = test.copy()
bro = bro.sort_values(by=["vesselId", "time"]).reset_index(drop=True)
worker["ID"]=bro["ID"]
worker = worker.sort_values(by="ID").reset_index(drop=True)

print(time.time()-start)
worker.tail()

655.084874868393


delta_time   last_lat    last_lon  speed_lag   latitude   longitude  \
51734      1355.0  37.169678  -76.312865   4.048602  37.192217  -76.338260   
51735      1243.0  32.764916 -111.748337   0.422851  32.766429 -111.754568   
51736      1240.0  29.549560 -117.117265   9.773401  29.581438 -117.118436   
51737      1248.0  57.120175   17.517097   8.099219  57.149167   17.491327   
51738      1247.0  56.072109   10.621154   3.483858  56.074151   10.612243   

                       vesselId                time     ID  
51734  61e9f3a8b937134a3c4bfdf3 2024-05-12 23:59:58  51734  
51735  61e9f3b4b937134a3c4bfe77 2024-05-12 23:59:58  51735  
51736  61e9f46cb937134a3c4c02b7 2024-05-12 23:59:58  51736  
51737  61e9f465b937134a3c4c0269 2024-05-12 23:59:58  51737  
51738  61e9f3adb937134a3c4bfe39 2024-05-12 23:59:58  51738

In [15]:
ramg = [i for i in range(51739)]
res3 = [ramg,worker["longitude"],worker["latitude"]]
with open('res.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(["ID","longitude_predicted","latitude_predicted"])
    write.writerows(zip(*res3))
df = pd.read_csv('res.csv')
df.to_csv('res.csv', index=False)